In [4]:
# Libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import time
import random
import sklearn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from torch import nn
from torch import optim
from tqdm import tqdm
import torch.nn.functional as F

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Remove printing error
pd.options.mode.chained_assignment = None

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from sklearn import metrics
from sklearn.metrics import roc_auc_score

import shap

from functions_folder.genetic import *

In [2]:
# Set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


### Data

In [5]:
# Data
mimic_data = pd.read_csv('mimic_data.csv')


# Split into 'preprocessing dataset' and a 'train/vaid/test dataset'
stays = mimic_data['stay_id'].unique()
import random
random.Random(0).shuffle(stays)
model_data2 = mimic_data.set_index("stay_id").loc[stays].reset_index()
n = round(0.5 * len(stays))
preprocessing_stays = stays[:n]
hold_out_stays = stays[n:]
preprocessing_data = model_data2[model_data2['stay_id'].isin(preprocessing_stays)]
hold_out_data = model_data2[model_data2['stay_id'].isin(hold_out_stays)]
preprocessing_data.drop(columns=['iv_treatment_length'], inplace=True)
preprocessing_data

,stay_id,date,iv_flag,first_po_flag,po_flag,blood pressure diastolic0,blood pressure diastolic1,blood pressure diastolic2,blood pressure diastolic3,blood pressure diastolic4,...,temperature14_current_stay_difference,temperature15_current_stay_difference,temperature16_current_stay_difference,temperature17_current_stay_difference,temperature18_current_stay_difference,temperature19_current_stay_difference,temperature20_current_stay_difference,temperature21_current_stay_difference,temperature_mean_current_stay_difference,temperature_std_current_stay_difference
0,39409066.0,2137-05-22,1.0,1.0,0,0.575521,0.519798,0.272727,0.700000,0.0,...,0.6,0.710059,0.600000,0.350276,-1.000000,-1.000000,0.467482,0.4,0.365151,0.075809
1,39409066.0,2137-05-23,1.0,1.0,0,0.448568,0.454810,0.454545,0.411765,0.0,...,0.4,0.410488,0.600000,0.551909,0.500693,0.439452,0.565014,0.4,0.482215,0.321606
2,30322661.0,2169-10-09,1.0,NaN,0,0.432292,0.446094,0.909091,0.304348,0.0,...,0.4,0.396435,0.600000,0.437195,0.526688,0.561250,0.666566,0.4,0.432688,0.363971
3,30322661.0,2169-10-10,1.0,1.0,0,0.361979,0.375081,0.545455,0.304348,0.0,...,0.4,0.396435,0.600000,0.437195,0.526688,0.561250,0.666566,0.4,0.432688,0.363971
4,30322661.0,2169-10-11,1.0,1.0,0,0.397135,0.378309,0.181818,0.500000,0.0,...,1.0,0.388391,0.642857,0.437195,0.526688,0.561250,0.358257,1.0,0.432688,0.664467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,37575956.0,2153-02-15,1.0,1.0,0,0.479167,0.536475,0.090909,0.875000,0.0,...,0.6,0.568117,0.600000,0.479335,-1.000000,-1.000000,0.454375,0.4,0.295362,0.681000
14516,37575956.0,2153-02-16,1.0,1.0,0,0.522135,0.525500,0.272727,1.000000,0.0,...,0.6,0.568117,0.600000,0.479335,-1.000000,-1.000000,0.454375,0.4,0.295362,0.681000
14517,37540542.0,2161-05-10,1.0,NaN,0,0.253906,0.237573,0.090909,-1.000000,0.0,...,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000
14518,37540542.0,2161-05-11,1.0,1.0,0,0.397135,0.370562,0.363636,0.304348,0.0,...,0.6,0.562664,0.528571,-1.000000,-1.000000,-1.000000,-1.000000,0.4,0.234579,0.425554


In [4]:
all_shap_importance = pd.read_csv('all_shap_importance.csv')

shap_list = all_shap_importance[all_shap_importance['feature_importance_vals'] >= 0.5].col_name.unique().tolist()
shap_list = ['stay_id', 'date', 'iv_flag', 'first_po_flag', 'po_flag'] + shap_list
preprocessing_data = preprocessing_data[shap_list]
preprocessing_data

,stay_id,date,iv_flag,first_po_flag,po_flag,blood pressure systolic17_current_stay,heart rate17,respiratory rate11_current_stay,blood pressure mean11_current_stay,o2 saturation pulseoxymetry3_current_stay,...,blood pressure systolic12_current_stay,o2 saturation pulseoxymetry1_current_stay_difference,gcs - verbal response14_current_stay,temperature7_current_stay,temperature6_current_stay_difference,respiratory rate3,heart rate21_current_stay,o2 saturation pulseoxymetry7_difference,gcs - verbal response2_difference,heart rate8
0,39409066.0,2137-05-22,1.0,1.0,0,0.966557,-1.000000,0.761556,0.762332,1.000000,...,0.222222,0.542862,0.0,0.000000e+00,0.4,0.633333,0.000000,0.491376,-1.0,0.384615
1,39409066.0,2137-05-23,1.0,1.0,0,0.431357,0.343572,0.126674,0.539524,0.573477,...,0.333333,0.599999,0.0,1.083738e-28,0.4,0.745098,0.111111,0.279022,-1.0,0.615385
2,30322661.0,2169-10-09,1.0,NaN,0,0.414066,0.230392,0.095854,0.400227,0.583127,...,0.111111,0.599999,0.0,9.099612e-29,0.4,0.527778,0.148148,0.364522,-1.0,0.076923
3,30322661.0,2169-10-10,1.0,1.0,0,0.327966,0.221895,0.046076,0.280141,0.460829,...,0.333333,0.599999,0.0,0.000000e+00,0.4,0.275362,0.259259,0.364522,-1.0,0.153846
4,30322661.0,2169-10-11,1.0,1.0,0,0.248910,0.521848,0.032439,0.198931,0.430108,...,0.333333,0.599999,0.0,1.622159e-01,0.4,0.452381,0.592593,0.571676,-1.0,0.230769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,37575956.0,2153-02-15,1.0,1.0,0,0.988171,-1.000000,0.953186,0.569251,-1.000000,...,0.000000,0.142857,-1.0,-1.000000e+00,0.4,0.791667,0.000000,0.439876,0.0,0.230769
14516,37575956.0,2153-02-16,1.0,1.0,0,0.626321,-1.000000,0.166640,0.800980,0.774194,...,0.111111,0.142857,-1.0,-1.000000e+00,0.4,0.904762,0.111111,0.439876,0.0,0.538462
14517,37540542.0,2161-05-10,1.0,NaN,0,0.968004,-1.000000,0.761556,0.718315,-1.000000,...,0.111111,-1.000000,-1.0,-1.000000e+00,-1.0,-1.000000,0.000000,-1.000000,-1.0,-1.000000
14518,37540542.0,2161-05-11,1.0,1.0,0,0.606376,0.521273,0.387155,0.412453,0.564516,...,0.333333,0.085715,0.0,2.895331e-29,-1.0,0.263889,0.259259,-1.000000,-1.0,0.307692


### Genetic

In [5]:
start = time.time()
whole_population = main_genetic_loop(50, preprocessing_data, max_iter=10, max_features=5, mutate_bool=False, verbose=True)
print('Time elapsed for executing the recursive GA: ', time.time() - start, ' seconds')




ITERATION NUMBER 1 (Iteration max = 10)



GENERATING RANDOM INDIVIDUALS.... 
Generated list of 50 individuals
Calculating auroc for individual  111011000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


100%|██████████| 9/9 [00:01<00:00,  4.74it/s]


mimic test AUROC result: 0.7907009211283822


100%|██████████| 20/20 [00:04<00:00,  4.59it/s]


eicu Test AUROC result: 0.579283241869422
auroc for the classifier trained for individual 111011000000000000000000000000000000000000000000000000000000000000000000000000000000000000000:  0.7907009211283822
Calculating auroc for individual  111110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


...

eicu Test AUROC result: 0.5
auroc for the classifier trained for individual 100000010000001010111010101101111111000101001110001001000101011000100001011101010000000101000:  0.8080195739781232
Calculating auroc for individual  101001110001100000010100001011001111000101000001001011000111110001101001001011011001111111101


100%|██████████| 9/9 [00:01<00:00,  4.92it/s]


mimic test AUROC result: 0.8145898100172712


100%|██████████| 20/20 [00:04<00:00,  4.79it/s]

eicu Test AUROC result: 0.49898164449295634
auroc for the classifier trained for individual 101001110001100000010100001011001111000101000001001011000111110001101001001011011001111111101:  0.8145898100172712
Generated population list (with weights): [('011001000100010010110100001001001000010000110011100010010001000001100000111011000010000110011', 1.9547028947482024), ('001110011110111101111101000101001101101101000010110000010111110100100111100101010101110011011', 2.000698796405624), ('011001100100110110111100001001000100010000100010100000010011010100100101001011110000110100000', 1.9828348009576346), ('011100001100000011010000101001000100010000100010100010110001000001000011010111011001100011011', 1.9339754537639233), ('011010010100010100011110010001001010100101000101001000000001011001111010001001010110100111010', 2.0179156242387193), ('001000111100001111101101100110101000011001100110011100000111000111111001101011001010000111100', 1.9862731048781037), ('10110101100001011110011010101100000

In [6]:
best_population_dict = {}
best_population_list = []
for key, value in whole_population.items():
    max_score = -99
    #sub_list = []
    for i in range(len(value)):
        if value[i][1] > max_score:
            max_score = value[i][1]
            max_feature_set = value[i][0]
        #elif value[i][1] >= max:
        #    max_score2 = value[i][1]
        #    max_feature_set2 = value[i][0]
    sub_tuple = (max_feature_set, max_score)
    #sub_list.append(sub_tuple)
    #for x in sub_list:
    best_population_list.append(sub_tuple)
    best_population_dict[key] = [sub_tuple]

In [7]:
best_population_dict
best_population_list


{'1': [('110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
   0.8019933793897525)],
 '2': [('110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
   0.799961139896373)],
 '3': [('101010100000000000000000000000000000001001000000010000000000011000000000000000000000100000000',
   0.8051683937823834)],
 '4': [('100111010000000100001000100011000000000001000010010100100110111000000000000000000000100000000',
   0.8132124352331607)],
 '5': [('100110111100000100001000100011000000010001000010010100100110111000000000000000000000101000100',
   0.8068206678180769)],
 '6': [('101111010000010000001000100011000000010001010011010100100110111000000000000000001000101100001',
   0.8077533103051238)],
 '7': [('001001110010000000001100000000011001101111010111101100000000101111010010010000100000001101100',
   0.8160693724812895)],
 '8': [('1011100001000001101101000010110011000001010101110010100001110110011010011010110111

[('110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
  0.8019933793897525),
 ('110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
  0.799961139896373),
 ('101010100000000000000000000000000000001001000000010000000000011000000000000000000000100000000',
  0.8051683937823834),
 ('100111010000000100001000100011000000000001000010010100100110111000000000000000000000100000000',
  0.8132124352331607),
 ('100110111100000100001000100011000000010001000010010100100110111000000000000000000000101000100',
  0.8068206678180769),
 ('101111010000010000001000100011000000010001010011010100100110111000000000000000001000101100001',
  0.8077533103051238),
 ('001001110010000000001100000000011001101111010111101100000000101111010010010000100000001101100',
  0.8160693724812895),
 ('101110000100000110110100001011001100000101010111001010000111011001101001101011011100000111000',
  0.8098704663212435),
 ('1011011100011000000101

In [8]:
best_score = 0
for i in range(len(best_population_list)):
    if best_population_list[i][1] > best_score:
        best_score = best_population_list[i][1]
        best_feature_set = best_population_list[i][0]
print(best_score)
print(best_feature_set)

0.8175460564191135
110111100011000100010101000000001011100101111010000000001100010100101000011100000010101110001


In [10]:
# Best short
110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
# Best long
110111100011000100010101000000001011100101111010000000001100010100101000011100000010101110001

110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

110111100011000100010101000000001011100101111010000000001100010100101000011100000010101110001

In [15]:
# Get columns
X_data = preprocessing_data.drop(columns=['stay_id', 'po_flag', 'date', 'iv_flag', 'first_po_flag'])

# Best short
print('Best short')
# Filter for features in this individual
individual = '110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'
X_data.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]].columns
len(X_data.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]].columns)
# Best long
print('Best long')
# Filter for features in this individual
individual = '110111100011000100010101000000001011100101111010000000001100010100101000011100000010101110001'
X_data.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]].columns
len(X_data.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]].columns)

Best short


Index(['blood pressure systolic17_current_stay', 'heart rate17',
       'blood pressure mean11_current_stay',
       'o2 saturation pulseoxymetry3_current_stay',
       'gcs - motor response15_current_stay'],
      dtype='object')

5

Best long


Index(['blood pressure systolic17_current_stay', 'heart rate17',
       'blood pressure mean11_current_stay',
       'o2 saturation pulseoxymetry3_current_stay',
       'gcs - motor response15_current_stay', 'heart rate17_current_stay',
       'gcs - verbal response10_current_stay', 'heart rate14',
       'o2 saturation pulseoxymetry17', 'gcs - motor response10_current_stay',
       'respiratory rate17_current_stay', 'blood pressure mean5_current_stay',
       'respiratory rate6_current_stay_difference',
       'temperature7_current_stay_difference',
       'blood pressure systolic15_current_stay',
       'heart rate9_current_stay_difference',
       'blood pressure systolic5_difference', 'heart rate3',
       'o2 saturation pulseoxymetry9', 'o2 saturation pulseoxymetry2',
       'gcs - motor response_std_current_stay',
       'gcs - verbal response7_current_stay', 'gcs - motor response13',
       'heart rate16_current_stay', 'gcs - motor response0_current_stay',
       'temperature10_

37

In [9]:
# Save
'''with open('genetic_short_whole_population.pkl', 'wb') as f:
    pickle.dump(whole_population, f)
with open('genetic_short_best_population_dict.pkl', 'wb') as f:
    pickle.dump(best_population_dict, f)
with open('genetic_short_best_population_list.pkl', 'wb') as f:
    pickle.dump(best_population_list, f)'''